In [ ]:
import pandas as pd
import numpy as np
from typing import Optional, Final

In [ ]:
data = pd.read_excel("статистика_старт.xlsx")
data

In [ ]:
features: tuple[str] = ("Регион",
                        "Доходы гостиниц и аналогичных средств размещения",
                        "Число гостиниц и средств размещения",
                        "Численность размещенных лиц в гостиницах и ананлогичных средствах размещения",
                        "Число ночевок в гостиницах и ананлогичных средствах размещения",
                        "Средняя численность работников, включая внешних совместителей",
                        "Число туристских фирм",
                        "Средства, поступившие от реализации туристкого продукта",
                        "Число реализованных турпакетов населению",
                        "Индекс стоимости жизни",
                        "Год")
years: Final[list[int]] = list(range(2002, 2020))
years.pop(1)

Собираем все упоминаемые названия регионов

In [ ]:
regions: set[str] = set()
mess: list[str] = data["Unnamed: 0"].fillna("").to_list()
mess.append("")
print(mess)

In [ ]:
def remove_numbers(string: str) -> str:
    if string[0].isdigit():
        space_position: int = string.find(" ")
        string = string[space_position + 1:]
    return string

features_positions: list[int] = list()
i: int = 0
string = mess[i]
while string != features[-2]: 
    if string == "":
        i += 1
        string = mess[i]
        continue
    if string in features:
        features_positions.append(i)
        i += 1
        string = mess[i]
        while string != "":
            regions.add(remove_numbers(string.strip()))
            i += 1
            string = mess[i]
features_positions.append(i)
regions = sorted(list(regions))
print(*enumerate(regions), sep="\n")

In [ ]:
new_data: pd.DataFrame = pd.DataFrame(data=np.full((len(regions) * len(years), len(features)), None), columns=features)
for i in range(len(regions)):
    for j, year in enumerate(years):
        new_data.iloc[i * len(years) + j].iloc[0] = regions[i]
        new_data.iloc[i * len(years) + j].loc["Год"] = year
new_data.set_index(["Регион", "Год"], inplace=True)

new_data

In [ ]:
cities = set()
i += 1
string = mess[i]
while string != "":
    cities.add(remove_numbers(string.strip()))
    i += 1
    string = mess[i]

cities

In [ ]:
cities_to_regions: dict[str, Optional[str]] = {"Абакан": regions[10],
                                               "Барнаул": regions[1],
                                               "Горно-Алтайск": regions[7],
                                               "Иркутск": regions[3],
                                               "Кемерово": None, # Кемеровская область не упоминается
                                               "Красноярск": regions[4],
                                               "Кызыл": regions[9],
                                               "Новосибирск": regions[5],
                                               "Омск": regions[6],
                                               "Северобайкальск": regions[8],
                                               "Томск": regions[12],
                                               "Тюмень": regions[13],
                                               "Улан-Удэ": regions[8],
                                               "Ханты-Мансийск": regions[15],
                                               "Юрга": None # Кемеровская область не упоминается
                                               }

In [ ]:
'''ities_to_regions: dict[str, dict[str]] = {regions[10]: {"Абакан"},
                                           regions[1]: {"Барнаул"},
                                           regions[7]: {"Горно-Алтайск"},
                                           regions[3]: {"Иркутск"},
                                           regions[4]: {"Красноярск"},
                                           regions[9]: {"Кызыл"},
                                           regions[5]: {"Новосибирск"},
                                           regions[6]: {"Омск"},
                                           regions[8]: {"Северобайкальск", "Улан-Удэ"},
                                           regions[12]: {"Томск"},
                                           regions[13]: {"Тюмень"},
                                           regions[15]: {"Ханты-Мансийск"},
                                           regions[0]: {},
                                           regions[2]: {},
                                           regions[11]: {},
                                           regions[14]: {},
                                           regions[16]: {},
                                           regions[17]: {}
                                           }'''

In [ ]:
for feature_index, position in enumerate(features_positions[:-1]):
    position += 1
    string = mess[position].strip()
    while string != "":
        region = remove_numbers(string)
        feature = features[feature_index + 1]
        for i, year in enumerate(years, start=1):
            new_data.loc[(region, year), feature] = data.iloc[position].iat[i]
        position += 1
        string = mess[position].strip()

new_data

In [ ]:
cities_data = data.iloc[-len(cities):,:]
cities_data.fillna(0)


In [ ]:
for _, row in cities_data.iterrows():
    city = remove_numbers(row.iloc[0].strip())
    for i, item in enumerate(row):
        region = cities_to_regions[city]
        if region != None:
            for j, year in enumerate(years, start=1):
                new_data.loc[(region, year), features[-2]] = row.iat[j]

new_data

In [ ]:
new_data = new_data.applymap(lambda x: 0 if str(x).strip() == "" else x, na_action="ignore")
new_data.fillna(0, inplace=True)
new_data